# Ising Simulation

This class is written to accept $J$ matrices and perform 2D Ising simulations on  a square lattice with $N^2$ sites.



# Import functions and modules

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import fmin, minimize

import sys
sys.path.append("..") # Adds higher directory to python modules path.

from src.isingsim import IsingSim, compress_histogram, load_club_list, sd2_loss, loss_func

In [18]:
#Needed for histogram compression - a list of configurations to 'club together' due to symmetry
# You can define your own if you wish.
club_list = load_club_list('../data/club_list.txt')

# Working Section

Here we define the model. We are going with a random bond model for nearest neighbors only. This is defined by scipy distributions to sample from

In [19]:
#define the model. You should supply a J_mat, like the one below

J_mat = [[0,0,0,0,0],
         [0,0.0,1.0,0,0],
         [0,1.0,0,1.0,0],
         [0,0,1.0,0,0],
         [0,0,0,0,0]
         ]

#Instantiate Model
ising_model =  IsingSim(N=20, J_mat = np.array(J_mat), save_trajectories=True, T = 2.5, 
                        eqSteps =500, mcSteps = 500)

## Run the simulation

It should take five minutes or so. Need to explore ways to speed this up. Cython should be promising.

In [ ]:
ising_model.performIsingSim()

  1%|          | 5/500 [00:00<00:11, 42.56it/s]


---Performing Equlibration---



  1%|          | 3/500 [00:00<00:24, 20.62it/s]


---Finished...
----Performing MC Moves----



  6%|▌         | 31/500 [00:01<00:24, 19.54it/s]

## Plot the results

In [ ]:
results = ising_model.results
print("\nEnergy: {0:2f}\nSusceptibility: {1:2f} \nMagnetization {2:2f} \nSpecific Heat: {3:2f}\n".format(
    results['Energy'], 
      results['Susceptibility'], 
      results['Magnetization'], 
      results['SpecificHeat']))

config = results['config']
histogram = results['Histogram']

plt.figure()
plt.imshow(config[-1,:,:])
plt.colorbar()

plt.figure(0)
plt.bar(np.arange(len(histogram)), histogram)

# Do some optimization

In [ ]:
#let's see if we can find this with optimization, starting with J = 0.55
#Here we use a Newton method

output = minimize(loss_func, x0 = [0.55], 
                  args = (ising_model.T,histogram, club_list), 
                  options = {'maxiter':3, 'disp':True, 'eps': 1E-1}, method = 'TNC', bounds = ((-1,1),))